In [3]:
import pickle
from operator import itemgetter
import pandas as pd
import plotly.graph_objects as go

from hydra.plotly_charts import PlotlyPriceChart, supersample_data
from hydra.utils import split_dupes

filename = "../2020-05-01 1d count=365 overlap=18h detrend=True.enviro"
file = None
with open(filename, "rb") as handle:
    file = pickle.load(handle)
interesting, charts, aggregate, inputs = itemgetter('data', 'charts', 'aggregate', 'inputs')(file)
timecap = pd.to_timedelta(inputs['timecap'])
overlap = pd.to_timedelta(inputs['overlap'])
traceCount = int(1.0/((timecap-overlap)/timecap))


pair = "XBTUSD"
startDate = charts[0]['startDate']
endDate = charts[-1]['endDate']
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"
print(startDate, endDate)
trendlines = {i:[] for i in range(traceCount)}
ifft_prices = {i:[] for i in range(traceCount)}
for index, chart in enumerate(charts):
    idx = index % traceCount
    trendlines[idx].append(chart['trendline'])
    ifft_prices[idx].append(chart['majority_price_proportion'])

# print(trendlines)
priceChart = PlotlyPriceChart(pair, startDate, endDate)

def add_trace(priceChart, data, chartParams={}, interval=1):
    d = pd.concat(data)
    supersampled = supersample_data(d, interval)
    priceChart.add_trace( 
        go.Scatter(x=supersampled[1440].index, y=supersampled[1440].values, **chartParams), 
        supersampled, 
        {"x": "index", "y": "values"},
        startDate,
        endDate
    )
    

for index in range(traceCount):
    add_trace(priceChart, trendlines[index], {"mode":"markers", "name":f"trend-{index}"})
    add_trace(priceChart, ifft_prices[index], {"mode":"lines", "name":f"ifft-{index}"})

priceChart.figure


2020-05-01 00:00:00 2020-08-01 00:00:00


FigureWidget({
    'data': [{'close': array([ 9004.7    ,  8824.6    ,  8854.9    ,  8983.1    ,  9052.6    ,
…